In [99]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

# Filtering

In [100]:
import numpy as np 
from numpy import matrix

def forward_1(PI, A, B, Observables):
    #rows = [0] * len(Observables)
    alpha_matrix = []
    curr = []
    for t,t_value in enumerate(Observables):
        nex = [0] * len(A)
        for a,a_value in enumerate(A): 
            if t == 0:
                nex[a] = PI[a] * B[a][t_value]
            else:
                for i in range(len(curr)):
                    nex[a] = sum(curr[i]*A[i][l] for l in range(len(A))) * B[a][t_value]
            
        alpha_matrix.append(nex)
        curr = nex
        
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [113]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    alpha_matrix = []
    curr = [0] *len(A)
    #nex = [0] * len(A)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
    alpha_matrix.append(curr)
    for t in range (1, len(Observables)):
        nex = [0] * len(A)
        for i in range (0, len(A)):
            for j in range (0, len(A)):
                #nex = alpha_matrix[t-1][j]
                t_value = Observables[t]
                nex[i] =  nex[i] + curr[j] * A[j][i]
                nex[i] = nex[i] * B[i][t_value] 
        alpha_matrix.append(nex)
        curr = nex
            
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [138]:
def forward_normalized_1(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    c = [0] * len(Observables)
    for i,i_value in enumerate(alpha):
        for j,j_value in enumerate(i_value):
            c[j] += i_value[j]
    
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha[i][j] = alpha[i][j] * (1/c[j])
    
    return alpha, c

In [104]:
def forward_normalized(PI, A, B, Observables):
    alpha_matrix = []
    curr = [0] *len(A)
    c = [0] * len(Observables)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
        c[0] = c[0] + curr[i]
    alpha_matrix.append(curr)
    
    c[0] = 1/c[0]
    for i in range (0, len(A)):
        alpha_matrix[0][i] = c[0] * alpha_matrix[0][i]
    
    for t in range (1, len(Observables)):
        nex = [0] * len(A)
        c[t] = 0
        for i in range (0, len(A)):
            for j in range (0, len(A)):
                #nex = alpha_matrix[t-1][j]
                t_value = Observables[t]
                nex[i] =  nex[i] + curr[j] * A[j][i]
                nex[i] = nex[i] * B[i][t_value] 
            c[t] = c[t] + nex[i]
            
        c[t] = 1/c[t]
        for i in range (0,len(A)):
            nex[i] = c[t] * nex[i]
        alpha_matrix.append(nex)
        curr = nex
        
        
            
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix,c

In [120]:
alpha = forward(PI, A, B, Observables)

In [121]:
alpha

[[0.25, 0.025],
 [0.0020025000000000004, 0.006995000000000001],
 [0.0004039975000000001, 0.0003504015000000001],
 [3.2670201500000015e-06, 7.324219970000003e-05]]

In [122]:
c

array([2.50000000e-02, 6.99500000e-03, 3.50401500e-04, 7.32421997e-05])

In [139]:
alpha_unorm = forward_1(PI, A, B, Observables)

In [140]:
alpha_unorm

[[0.25, 0.025],
 [0.0025000000000000005, 0.005000000000000001],
 [0.0025000000000000005, 0.00025000000000000006],
 [2.5000000000000008e-05, 5.0000000000000016e-05]]

# Evaluation

In [125]:
def evaluation_unnormalized(alpha):
    probability = 0
    for i in alpha:
        for j in i:
             probability += j        
    return probability

In [126]:
evaluation_unnormalized(alpha_unorm)

0.285325

In [127]:
evaluation_unnormalized(alpha)

0.28482840821985

In [129]:
def evaluation_normalized(alpha, c):
    alpha_norm = alpha.copy()
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha_norm[i][j] = alpha[i][j] * c[j]
    
    probability = 0
    for i in alpha_norm:
        for j in i:
             probability += j 
    return probability

In [130]:
def evaluation_normalized(alpha, c):
    probability = 0
    for i in alpha:
        for j in i:
             probability += j
    return probability 

In [132]:
alpha_norm,c = forward_normalized(PI, A, B, Observables)

In [133]:
alpha_norm

[[0.9090909090909091, 0.09090909090909091],
 [0.22256182272853567, 0.7774381772714642],
 [0.535522316439974, 0.46447768356002594],
 [0.04270099938811492, 0.957299000611885]]

In [134]:
c

[3.6363636363636362, 30.564045568213384, 11.926712522153396, 9.860236471879276]

In [141]:
alpha_norm_1, c_1 = forward_normalized_1(PI, A, B, Observables)

In [142]:
alpha_norm_1

[[0.9904529663314288, 0.7711611945144807],
 [0.007933528260314746, 0.21577090222515175],
 [0.0016005620890619258, 0.010808641571986636],
 [1.2943319194528203e-05, 0.0022592616883808066]]

In [143]:
c_1

[0.25240976452015, 0.0324186436997, 0, 0]

# Backward Probability

In [97]:
def backward(PI, A, B, Observables):
    
    beta = []
    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return beta #no of initial states * length of observation sequence

In [98]:
backward(PI, A, B, Observables)

[[0.014028409100000003, 0.0005229484545000001],
 [0.03481800000000001, 0.01012091],
 [0.41000000000000003, 0.05045],
 [1, 1]]

In [102]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
    beta = backward(PI, A, B, Observables)


    for i in range(len(beta)):
        for j in range(len(beta[i])):
            beta[i][j] = beta[i][j] * (1/c[j])
            
    return beta

In [101]:
backward_normalized(PI, A, B, Observables)

[[0.003577595030727501, 1.5845338171350004e-05],
 [0.008879460450000003, 0.00030666357300000003],
 [0.10456025000000001, 0.0015286350000000002],
 [0.255025, 0.030300000000000004]]

# Smoothing by Forward-Backward Algorithm

In [102]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    return matrix